## Imports

In [2]:
import logging
import os
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import googlemaps
from google.oauth2 import service_account
from googleapiclient.discovery import build
import numpy as np
from dotenv import load_dotenv

In [3]:
# Load environment variables from the .env file
load_dotenv(".env")

True

## Set up Logging

In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('scrape_google_maps_reviews.log', mode='w'),  # Log to file
        logging.StreamHandler(sys.stdout)  # Log to console (for Jupyter/real-time output)
    ]
)

## Read Addresses from Google Sheet

In [5]:
# Load credentials from the JSON key file
SERVICE_ACCOUNT_FILE = '/Users/madhumithakumar/Documents/bgorg_clients/google_sheets_service_account_key.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

# Google Sheets ID and Range
SPREADSHEET_ID = '11-bAMl8nkpcQK1uxoRZ2pCZ1AKm70Wuu56McK0Uk5PU'
RANGE_NAME = 'NailsnBeyond!A1:I11'  # Adjust as needed

# Load Google Sheets data into a DataFrame
def fetch_data_from_gsheet():
    try:
        # Build the Google Sheets API service
        service = build('sheets', 'v4', credentials=credentials)
        sheet = service.spreadsheets()

        # Fetch data from the specified range
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
        values = result.get('values', [])

        # Convert the fetched data to a DataFrame
        if values:
            df = pd.DataFrame(values[1:], columns=values[0])  # First row as header
        else:
            df = pd.DataFrame()

        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

In [6]:
df = fetch_data_from_gsheet()

2024-12-28 22:21:26,110 - INFO - file_cache is only supported with oauth2client<4.0.0


In [7]:
#df.set_index("BGorgeous_Client_Name", inplace=True)

In [8]:
#business_name, address = df.loc["Naturals_Nookampalayam", ["Business Name", "Address"]]

In [9]:
#business_name

In [10]:
#address

In [11]:
df.head()

,City,Area,Address,Business Name,Status,Franchise Owner,Subscription,BGorgeous_Client_Name,Phone
0,Chennai,Royapettah,"No 114, 1st Floor, Cathedral Rd, Gopalapuram, ...",Nails n Beyond,None,None,None,None,None
1,Chennai,Velachery,"142 Phoenix Market City, Road, Indira Gandhi N...",Nails 'n Beyond,None,None,None,None,None
2,Chennai,Nungambakkam,"3rd Floor, Ispahani Centre, 123 & 124, Nungamb...",Nails N Beyond,None,None,None,None,None
3,Chennai,Anna Nagar West,"No S68, 2nd Floor, VR Chennai, Anna Nagar West...",Nails 'n Beyond,,,,,919003179191
4,Chennai,Alwarpet,"No 66, Oxford Center, Subbaraya Avenue, CP Ram...",Nails 'n Beyond,,,,,914424661515


## Places API

In [12]:
api_key = os.environ.get("GOOGLE_CLOUD_API_KEY")

In [13]:
def scrape_google_maps_reviews(api_key, business_name, city_name, max_locations):
    """
    Extract metadata for up to 200 locations using pagination in the Google Places API.
    Extract all reviews into a single CSV file and save Google Maps URLs to a text file.

    Parameters:
        api_key (str): Google API key.
        business_name (str): Business name to query.
        city_name (str): City name for the search.
        max_locations (int): Total number of locations to extract.

    Returns:
        None: Saves the data to CSV files and URLs to a text file.
    """
    logging.info("Starting the scrape_google_maps_reviews function")
    gmaps = googlemaps.Client(key=api_key)
    location_data = []
    all_reviews = []
    urls = []  # To store the URLs

    try:
        processed_locations = 0
        next_page_token = None

        while processed_locations < max_locations:
            # Fetch places using the API
            query = f"{business_name}, {city_name}"
            if next_page_token:
                response = gmaps.places(query=query, page_token=next_page_token)
            else:
                response = gmaps.places(query=query)

            # Handle API errors
            if response.get("status") != "OK":
                logging.error(f"Places API failed: {response.get('status')}")
                break

            # Process results
            results = response.get("results", [])
            for place in results:
                if processed_locations >= max_locations:
                    break

                place_id = place["place_id"]
                name = place.get("name", "N/A")
                address = place.get("formatted_address", "N/A")
                rating = place.get("rating", "N/A")
                location_id = f"LOC_{processed_locations + 1}"
                user_ratings_total = place.get("user_ratings_total", 0)

                # Generate Google Maps URL
                url = f"https://www.google.com/maps/place/?q=place_id:{place_id}"
                urls.append(url)  # Add URL to list

                location_data.append({
                    "Location ID": location_id,
                    "City Area": city_name,
                    "Name": name,
                    "Address": address,
                    "Rating": rating,
                    "Total Reviews": user_ratings_total,
                    "Place ID": place_id,
                    "Google Maps URL": url
                })

                # Fetch reviews for the current location
                try:
                    place_details = gmaps.place(place_id=place_id, fields=["reviews"])
                    reviews = place_details.get("result", {}).get("reviews", [])

                    for review in reviews:
                        reviewer_name = review.get("author_name", "N/A")
                        review_text = review.get("text", "No text")
                        review_rating = review.get("rating", "N/A")
                        review_timestamp = review.get("time")
                        review_date_actual = datetime.utcfromtimestamp(review_timestamp).strftime('%Y-%m-%d %H:%M:%S') if review_timestamp else "N/A"

                        all_reviews.append({
                            "Location ID": location_id,
                            "City Area": city_name,
                            "Business Name": name,
                            "Address": address,
                            "Reviewer Name": reviewer_name,
                            "Review": review_text,
                            "Rating": review_rating,
                            "Review Date": review_date_actual,
                            "Place ID": place_id
                        })

                except Exception as e:
                    logging.warning(f"Failed to fetch reviews for {name}: {e}")

                processed_locations += 1

            # Check for pagination token
            next_page_token = response.get("next_page_token")
            if not next_page_token:
                logging.info("No more pages available.")
                break

            logging.info("Waiting for next page token to activate...")
            time.sleep(3)  # Wait for the token to become active

        # Save location data to CSV
        pd.DataFrame(location_data).to_csv(f"data/location_data/naturals_{city_name}_data.csv", index=False)
        logging.info(f"Successfully saved {processed_locations} locations to f'data/{business_name}_{city_name}_data.csv'.")

        # Save all reviews to a single CSV
        if all_reviews:
            pd.DataFrame(all_reviews).to_csv(f"data/review_data/naturals_{city_name}_reviews.csv", index=False)
            logging.info(f"Successfully saved all reviews to f'data/{business_name}_{city_name}_all_reviews.csv'.")
        else:
            logging.warning("No reviews were found for any location.")

        # Save Google Maps URLs to a text file
        with open(f"data/location_data/maps_urls/{city_name}.txt", "w") as file:
            for url in urls:
                file.write(url + "\n")
        logging.info(f"Successfully saved Google Maps URLs to 'data/location_data/maps_urls/{city_name}.txt'.")

    except Exception as e:
        logging.error(f"An error occurred: {e}")

In [14]:
# scrape_google_maps_reviews(api_key, "Naturals Signature", "Chennai South", 100)

## Go from Google Sheet
Pass address directly

In [15]:
api_key = os.environ.get("GOOGLE_CLOUD_API_KEY")

In [16]:
def scrape_google_maps_metadata(api_key, business_name, city_name, max_locations):
    """
    Extract metadata for up to `max_locations` using pagination in the Google Places API,
    including latitude and longitude.

    Parameters:
        api_key (str): Google API key.
        business_name (str): Business name to query.
        city_name (str): City name for the search.
        max_locations (int): Total number of locations to extract.

    Returns:
        list[dict]: List of metadata dictionaries for the locations.
    """
    import googlemaps
    import logging
    import time

    logging.info("Starting the scrape_google_maps_metadata function")
    gmaps = googlemaps.Client(key=api_key)
    location_data = []

    try:
        processed_locations = 0
        next_page_token = None

        while processed_locations < max_locations:
            # Fetch places using the API
            query = f"{business_name}, {city_name}"
            if next_page_token:
                response = gmaps.places(query=query, page_token=next_page_token)
            else:
                response = gmaps.places(query=query)

            # Handle API errors
            if response.get("status") != "OK":
                logging.error(f"Places API failed: {response.get('status')}")
                break

            # Process results
            results = response.get("results", [])
            for place in results:
                if processed_locations >= max_locations:
                    break

                place_id = place["place_id"]
                name = place.get("name", "N/A")
                address = place.get("formatted_address", "N/A")
                rating = place.get("rating", "N/A")
                user_ratings_total = place.get("user_ratings_total", 0)

                # Extract latitude and longitude
                geometry = place.get("geometry", {}).get("location", {})
                lat = geometry.get("lat", "N/A")
                lng = geometry.get("lng", "N/A")

                # Generate Google Maps URL
                url = f"https://www.google.com/maps/place/?q=place_id:{place_id}"

                location_data.append({
                    "Location ID": f"LOC_{processed_locations + 1}",
                    "City Area": city_name,
                    "Name": name,
                    "Address": address,
                    "Latitude": lat,
                    "Longitude": lng,
                    "Rating": rating,
                    "Total Reviews": user_ratings_total,
                    "Place ID": place_id,
                    "Google Maps URL": url
                })

                processed_locations += 1

            # Check for pagination token
            next_page_token = response.get("next_page_token")
            if not next_page_token:
                logging.info("No more pages available.")
                break

            logging.info("Waiting for next page token to activate...")
            time.sleep(3)  # Wait for the token to become active

    except Exception as e:
        logging.error(f"An error occurred: {e}")

    return location_data

In [17]:
# scrape_google_maps_metadata(api_key, "Naturals Salon", "Adambakkam, Brindavan Nagar", 5)

In [18]:
# area, address = df.loc[2, ["Area", "Address"]]

In [19]:
# area

In [20]:
# address

In [21]:
df.head()

,City,Area,Address,Business Name,Status,Franchise Owner,Subscription,BGorgeous_Client_Name,Phone
0,Chennai,Royapettah,"No 114, 1st Floor, Cathedral Rd, Gopalapuram, ...",Nails n Beyond,None,None,None,None,None
1,Chennai,Velachery,"142 Phoenix Market City, Road, Indira Gandhi N...",Nails 'n Beyond,None,None,None,None,None
2,Chennai,Nungambakkam,"3rd Floor, Ispahani Centre, 123 & 124, Nungamb...",Nails N Beyond,None,None,None,None,None
3,Chennai,Anna Nagar West,"No S68, 2nd Floor, VR Chennai, Anna Nagar West...",Nails 'n Beyond,,,,,919003179191
4,Chennai,Alwarpet,"No 66, Oxford Center, Subbaraya Avenue, CP Ram...",Nails 'n Beyond,,,,,914424661515


In [22]:
len(df)

10

In [23]:
def process_dataframe(api_key, df):
    """
    Loop through a DataFrame and call `scrape_google_maps_metadata` for each row.

    Parameters:
        api_key (str): Google API key.
        df (pd.DataFrame): DataFrame containing 'Business Name' and 'City Name' columns.

    Returns:
        pd.DataFrame: A new DataFrame with the results.
    """
    import pandas as pd

    results = []

    for index, row in df.iterrows():
        city_name = row["City"] + " " + row["Area"]

        # Call the scrape_google_maps_metadata function
        data = scrape_google_maps_metadata(api_key, "Nails n Beyond", city_name, max_locations=20)  # Adjust max_locations as needed
        if data:
            results.extend(data)  # Extend the list with the results

    # Convert results to a DataFrame
    if results:
        return pd.DataFrame(results)
    else:
        print("No valid data retrieved.")
        return pd.DataFrame()  # Return an empty DataFrame if no results


In [24]:
# Process the DataFrame
result_df = process_dataframe(api_key, df)

2024-12-28 22:21:40,496 - INFO - Starting the scrape_google_maps_metadata function
2024-12-28 22:21:40,498 - INFO - API queries_quota: 60
2024-12-28 22:21:41,503 - INFO - No more pages available.
2024-12-28 22:21:41,509 - INFO - Starting the scrape_google_maps_metadata function
2024-12-28 22:21:41,510 - INFO - API queries_quota: 60
2024-12-28 22:21:42,404 - INFO - No more pages available.
2024-12-28 22:21:42,409 - INFO - Starting the scrape_google_maps_metadata function
2024-12-28 22:21:42,411 - INFO - API queries_quota: 60
2024-12-28 22:21:43,866 - INFO - No more pages available.
2024-12-28 22:21:43,871 - INFO - Starting the scrape_google_maps_metadata function
2024-12-28 22:21:43,873 - INFO - API queries_quota: 60
2024-12-28 22:21:45,109 - INFO - No more pages available.
2024-12-28 22:21:45,114 - INFO - Starting the scrape_google_maps_metadata function
2024-12-28 22:21:45,116 - INFO - API queries_quota: 60
2024-12-28 22:21:47,421 - INFO - No more pages available.
2024-12-28 22:21:47,

In [25]:
len(result_df)

25

In [26]:
result_df = result_df.drop_duplicates(subset=['Place ID'])

In [27]:
len(result_df)

10

In [28]:
result_df.head()

,Location ID,City Area,Name,Address,Latitude,Longitude,Rating,Total Reviews,Place ID,Google Maps URL
0,LOC_1,Chennai Royapettah,Nails n Beyond,"No 114, 1st Floor, Cathedral Rd, Gopalapuram, ...",13.047563,80.254357,4.8,373,ChIJu_BEcNhnUjoR1nBoaKnbQU8,https://www.google.com/maps/place/?q=place_id:...
1,LOC_2,Chennai Royapettah,Nails 'n Beyond,"Shop No. B1, 27 Express Mall.2, Club House Roa...",13.058911,80.264528,5.0,4,ChIJmzVJsNpnUjoRO-VbI3aMgn0,https://www.google.com/maps/place/?q=place_id:...
2,LOC_1,Chennai Velachery,Nails 'n Beyond,"142 Phoenix Market City, Road, Indira Gandhi N...",12.992709,80.217270,4.9,355,ChIJCbZckZlnUjoRBp3RFB6IV-E,https://www.google.com/maps/place/?q=place_id:...
4,LOC_2,Chennai Nungambakkam,Nails N Beyond,"3rd Floor, Ispahani Centre, 123 & 124, Nungamb...",13.058399,80.248142,2.3,3,ChIJGQF0CEJmUjoRdH35zqWvIeM,https://www.google.com/maps/place/?q=place_id:...
5,LOC_1,Chennai Anna Nagar West,Nails 'n Beyond,"No S68, 2nd Floor, VR Chennai, Anna Nagar West...",13.080524,80.197132,4.7,276,ChIJWXplDnFlUjoR89bQNddfMSU,https://www.google.com/maps/place/?q=place_id:...


In [29]:
result_df["Total Reviews"].sum()

1568

In [30]:
result_df["Rating"].isna().sum()

0

In [31]:
(result_df['Total Reviews'] == 0).sum()

1

In [32]:
result_df["Name"].value_counts()

Name
Nails 'n Beyond    8
Nails n Beyond     1
Nails N Beyond     1
Name: count, dtype: int64

In [33]:
# List of allowed values
not_allowed_names = ["Shree Naturals", "GREEN TRENDS PERUNGALATHUR", "Sun Naturals Unisex Salon And Spa", "Studieo7 Family Salon & Bridal Studio - Nanganallur",
                    "Allure Unisex Salon", "Isha", "Green Trends Unisex Hair & Style Salon", "Page 3 Luxury Salon Kilpauk", "Vinodhbamaa Makeup Artistry | Beautician Course in Chennai | Makeup Training Academy | Makeup course",
                    "Naturals Corporate Office"]

# Keep rows where 'name' is in the list of allowed values
result_df = result_df[~result_df['Name'].isin(not_allowed_names)]

In [34]:
len(result_df)

10

In [35]:
result_df["Name"].value_counts()

Name
Nails 'n Beyond    8
Nails n Beyond     1
Nails N Beyond     1
Name: count, dtype: int64

In [36]:
result_df['full_location'] = result_df['City Area'] + " " + result_df['Name']

In [37]:
result_df.to_csv("data/nailsnbeyond_locations_metadata.csv")
result_df['Google Maps URL'].to_csv('data/nailsnbeyond_maps_urls.txt', index=False)

In [38]:
result_df.head()

,Location ID,City Area,Name,Address,Latitude,Longitude,Rating,Total Reviews,Place ID,Google Maps URL,full_location
0,LOC_1,Chennai Royapettah,Nails n Beyond,"No 114, 1st Floor, Cathedral Rd, Gopalapuram, ...",13.047563,80.254357,4.8,373,ChIJu_BEcNhnUjoR1nBoaKnbQU8,https://www.google.com/maps/place/?q=place_id:...,Chennai Royapettah Nails n Beyond
1,LOC_2,Chennai Royapettah,Nails 'n Beyond,"Shop No. B1, 27 Express Mall.2, Club House Roa...",13.058911,80.264528,5.0,4,ChIJmzVJsNpnUjoRO-VbI3aMgn0,https://www.google.com/maps/place/?q=place_id:...,Chennai Royapettah Nails 'n Beyond
2,LOC_1,Chennai Velachery,Nails 'n Beyond,"142 Phoenix Market City, Road, Indira Gandhi N...",12.992709,80.217270,4.9,355,ChIJCbZckZlnUjoRBp3RFB6IV-E,https://www.google.com/maps/place/?q=place_id:...,Chennai Velachery Nails 'n Beyond
4,LOC_2,Chennai Nungambakkam,Nails N Beyond,"3rd Floor, Ispahani Centre, 123 & 124, Nungamb...",13.058399,80.248142,2.3,3,ChIJGQF0CEJmUjoRdH35zqWvIeM,https://www.google.com/maps/place/?q=place_id:...,Chennai Nungambakkam Nails N Beyond
5,LOC_1,Chennai Anna Nagar West,Nails 'n Beyond,"No S68, 2nd Floor, VR Chennai, Anna Nagar West...",13.080524,80.197132,4.7,276,ChIJWXplDnFlUjoR89bQNddfMSU,https://www.google.com/maps/place/?q=place_id:...,Chennai Anna Nagar West Nails 'n Beyond


### Read Data

In [39]:
import pandas as pd

df = pd.read_csv("data/nailsnbeyond_locations_metadata.csv")

In [40]:
len(df)

10

In [41]:
df["Total Reviews"].sum()

1568